In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import yagmail

# CONFIGURATION
# Email credentials
SENDER_EMAIL = 'crawnicles@gmail.com'
APP_PASSWORD = ''  # Use App Password, not regular Gmail password
RECIPIENT_SMS = '@vtext.com'  # Your Verizon number 
URL = "https://underdogrescuemn.com/adoptable-dogs/"


def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Comment this out if you want to see the browser
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


def check_for_dog(driver, dog_name):
    driver.get(URL)
    time.sleep(5)  # Let JS load

    page_text = driver.find_element(By.TAG_NAME, "body").text.lower()
    print("Page text snippet:", page_text[:300])  # Debugging
    return dog_name.lower() in page_text


def send_sms_alert(dog_name):
    print(f"📲 Sending SMS alert for {dog_name}...")
    yag = yagmail.SMTP(SENDER_EMAIL, APP_PASSWORD)
    yag.send(
        to=RECIPIENT_SMS,
        subject="",
        contents=f"🐶 {dog_name} is now listed on Underdog Rescue MN!"
    )
    print("✅ SMS sent!")


def monitor_site(dog_name="Remy", interval=300):
    driver = setup_driver()
    while True:
        try:
            print(f"Checking for {dog_name}...")
            if check_for_dog(driver, dog_name):
                send_sms_alert(dog_name)
                break
        except Exception as e:
            print("❗ Error:", e)
        time.sleep(interval)
    driver.quit()


monitor_site("buster", interval=10)  # Check every 5 minutes
